#### 연습문제
1. --- 품목 상세보기 링크도 추가해서 크롤링
2. --- 사이트내에서 전체페이지를 확인할 수 있는 태그가 있는지 검사하고 해당 태그에서 총 페이지를 추출한 후
- 전체 페이지를 크롤링 할 수 있는 프로그램으로 변경

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [9]:
# 크롤링 문서 요청해서 응답객체 반환
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()
# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")


In [10]:
## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
# ul class:prdList grid4
ul=bs_obj.find("ul",{"class":"prdList grid4"})
## 품목 1개를 담고 있는 div 태그 추출
## div class:box
prd_boxes = ul.findAll("div", {"class":"box"})

In [47]:
# box안에 들어 있는 1개의 상품에서 정보를 추출해서 dict형태로  반환하는 함수
def get_product_info(box) :
    p_tag = box.find("p",{"class":"name"})
    span = p_tag.find("span")
    link = p_tag.find('a')['href']
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = price_ul.findAll("span")
    # 최종 data 추출 후 출력
    
    return {"prd_name":span.text,"price":price_span[1].text,"sale_price":price_span[-1].text,"detail":"https://jolse.com/"+link}


In [48]:
# 전달된 url 페이지에 접근해서 해당페지의 전체 상품 데이터를 추출 한 후 각 상품마다 get_product_info()함수를 호출해서
# 각 상품에대한 추출 정보를 받아옴 - 들어온 각 상품 정보를 리스트에 저장 한 수 해당 반환
def get_page_products(url) :
    url=url
    html = urlopen(url)
    htmls = html.read()
    # print(htmls)
    bs_obj = BeautifulSoup(htmls,"html.parser")

    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid4"})
    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes = ul.findAll("div", {"class":"box"}) #1개 페이지의 전체 상품
    prod_info_list = [get_product_info(box) for box in prd_boxes] # 현재 함수를 통해 접속한 문서(url 인수)의 각 화장품 정보가 
    # 저장되어 있는 리스트
    return prod_info_list

In [49]:
#총 페이지 수 얻어오기
last_page = int(bs_obj.find("p",{"class":"last"}).find('a')['href'][-2::1])
last_page

12

In [57]:
#여러 페이지의 화장품 정보를 추출해서 df 에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" #페이지 번호를 제외한 공통 url 문자열

df_fin = pd.DataFrame()
for i in range(1,last_page+1) :
    # url 완성 :page번호를 추가 i 변수 값을 활용
    urlfin =url + str(i)
    page_product = get_page_products(urlfin)
    df = pd.DataFrame(page_product) #현재 url로 접속한 페이지의 대해 수집된 제품정보를 df로 생성(한개 페이지 제품)
    df_fin = pd.concat([df_fin,df],axis=0,ignore_index=True) 
    
df_fin

,prd_name,price,sale_price,detail
0,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 13.99,https://jolse.com//product/detail.html?product...
1,SON&PARK BEAUTY WATER 340ml,USD 29.40,USD 24.99,https://jolse.com//product/detail.html?product...
2,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,USD 24.00,USD 13.99,https://jolse.com//product/detail.html?product...
3,SOME BY MI Galactomyces Pure Vitamin C Glow To...,USD 34.00,USD 17.99,https://jolse.com//product/detail.html?product...
4,PURITO Centella Green Level Calming Toner 200ml,USD 25.20,USD 21.42,https://jolse.com//product/detail.html?product...
...,...,...,...,...
565,It's skin Aloe Relaxing Toner 150ml,USD 19.00,USD 16.15,https://jolse.com//product/detail.html?product...
566,It's skin Hyaluronic Acid Moisture Toner 150ml,USD 19.00,USD 16.15,https://jolse.com//product/detail.html?product...
567,Ciracle Base Toner pH5.6 105.5ml,USD 23.48,USD 19.96,https://jolse.com//product/detail.html?product...
568,[Ciracle] Anti-Blemish Toner 105.5ml,USD 22.25,USD 18.91,https://jolse.com//product/detail.html?product...


In [58]:
df_fin['detail'][2]

'https://jolse.com//product/detail.html?product_no=16241&cate_no=1019&display_group=1'

In [ ]:
#div class xans-element- xans-product xans-product-normalpaging df-base-paging
#href="?page=12"